In [3]:
# how many tokens are in GRC
# how many tokens are in fiction section of COCA

import sqlite3, spacy

grc_conn = sqlite3.connect('D:\\graded_readers_mwe.sqlite') # This creates the database file
grc_c = grc_conn.cursor() # Creates a cursor

grc_c.execute('SELECT COUNT(*) FROM corpus')
print(grc_c.fetchone()[0])

grc_c.execute('SELECT COUNT(*) FROM corpus WHERE f_nf = "F"')
print(grc_c.fetchone()[0])

erf_levels = (1,2,3,4,5,6)
for level in erf_levels:
    grc_c.execute('SELECT COUNT(*) FROM corpus WHERE erf_level = ?', (level,))
    print(level, grc_c.fetchone()[0])

grc_conn.close()

16448662
14595260
1 435439
2 2684321
3 4116282
4 4870339
5 3217517
6 1107167


In [3]:
# Should produce counts for the number of n-grams by ERF level

# Then it should also put these out into lists. I can basically do what I'm doing, 
# just append each list of ERF words to the end with a tag for which erf level

import sqlite3, math

grc_conn = sqlite3.connect('D:\\graded_readers_mwe.sqlite') # This creates the database file
grc_c = grc_conn.cursor() # Creates a cursor
grc_c.execute("CREATE INDEX IF NOT EXISTS ngram_count_IN ON ngrams (ngram_size, total_grc_freq)")
grc_c.execute("CREATE INDEX IF NOT EXISTS ngram_count_fic_IN ON ngrams (ngram_size, total_grc_fic_freq)")
grc_c.execute("CREATE INDEX IF NOT EXISTS ngram_count_erf_one_IN ON ngrams (ngram_size, erf_one_freq)")
grc_c.execute("CREATE INDEX IF NOT EXISTS ngram_count_erf_two_IN ON ngrams (ngram_size, erf_two_freq)")
grc_c.execute("CREATE INDEX IF NOT EXISTS ngram_count_erf_three_IN ON ngrams (ngram_size, erf_three_freq)")
grc_c.execute("CREATE INDEX IF NOT EXISTS ngram_count_erf_four_IN ON ngrams (ngram_size, erf_four_freq)")
grc_c.execute("CREATE INDEX IF NOT EXISTS ngram_count_erf_five_IN ON ngrams (ngram_size, erf_five_freq)")
grc_c.execute("CREATE INDEX IF NOT EXISTS ngram_count_erf_six_IN ON ngrams (ngram_size, erf_six_freq)")
grc_c.execute("CREATE INDEX IF NOT EXISTS ngram_count_erf_one_fic_IN ON ngrams (ngram_size, erf_one_fic_freq)")
grc_c.execute("CREATE INDEX IF NOT EXISTS ngram_count_erf_two_fic_IN ON ngrams (ngram_size, erf_two_fic_freq)")
grc_c.execute("CREATE INDEX IF NOT EXISTS ngram_count_erf_three_fic_IN ON ngrams (ngram_size, erf_three_fic_freq)")
grc_c.execute("CREATE INDEX IF NOT EXISTS ngram_count_erf_four_fic_IN ON ngrams (ngram_size, erf_four_fic_freq)")
grc_c.execute("CREATE INDEX IF NOT EXISTS ngram_count_erf_five_fic_IN ON ngrams (ngram_size, erf_five_fic_freq)")
grc_c.execute("CREATE INDEX IF NOT EXISTS ngram_count_erf_six_fic_IN ON ngrams (ngram_size, erf_six_fic_freq)")
grc_conn.commit()

genres = ("acad", "blog", "fic", "mag", "news", "spok", "tvm", "web")
for genre in genres:
    coca_filename = 'D:\\coca_' + genre + '.sqlite'
    coca_conn = sqlite3.connect(coca_filename)
    coca_c = coca_conn.cursor()
    coca_c.execute("CREATE INDEX IF NOT EXISTS twogram_IN ON ngrams (ngram_size, wordoneID, wordtwoID)")
    coca_c.execute("CREATE INDEX IF NOT EXISTS threegram_IN ON ngrams (ngram_size, wordoneID, wordtwoID, wordthreeID)")
    coca_c.execute("CREATE INDEX IF NOT EXISTS fourgram_IN ON ngrams (ngram_size, wordoneID, wordtwoID, wordthreeID, wordfourID)")
    coca_c.execute("CREATE INDEX IF NOT EXISTS fivegram_IN ON ngrams (ngram_size, wordoneID, wordtwoID, wordthreeID, wordfourID, wordfiveID)")
    coca_conn.commit()
    coca_conn.close()





################# Function to pull out info from COCA #####################
def get_coca_info(n_gram, ngram_size):
    genres = ("acad", "blog", "fic", "mag", "news", "spok", "tvm", "web")
    genre_counts = {}
    ngram = n_gram.split("_")
    genre_counts["total"] = 0
    for genre in genres:
        coca_filename = 'D:\\coca_' + genre + '.sqlite'
        coca_conn = sqlite3.connect(coca_filename)
        coca_c = coca_conn.cursor()
        if ngram_size == 2:
            coca_c.execute("SELECT freq FROM ngrams \
                    JOIN lexicon AS L1 ON ngrams.wordoneID = L1.ID \
                    JOIN lexicon AS L2 ON ngrams.wordtwoID = L2.ID \
                    WHERE ngram_size = ? AND L1.type_lc = ? AND L2.type_lc = ?", \
                            (ngram_size, ngram[0], ngram[1]))  
        elif ngram_size == 3:
            coca_c.execute("SELECT freq FROM ngrams \
                    JOIN lexicon AS L1 ON ngrams.wordoneID = L1.ID \
                    JOIN lexicon AS L2 ON ngrams.wordtwoID = L2.ID \
                    JOIN lexicon AS L3 ON ngrams.wordthreeID = L3.ID \
                    WHERE ngram_size = ? AND L1.type_lc = ? AND L2.type_lc = ? AND L3.type_lc = ?", \
                            (ngram_size, ngram[0], ngram[1], ngram[2]))
        elif ngram_size == 4:
            coca_c.execute("SELECT freq FROM ngrams \
                    JOIN lexicon AS L1 ON ngrams.wordoneID = L1.ID \
                    JOIN lexicon AS L2 ON ngrams.wordtwoID = L2.ID \
                    JOIN lexicon AS L3 ON ngrams.wordthreeID = L3.ID \
                    JOIN lexicon AS L4 ON ngrams.wordfourID = L4.ID \
                    WHERE ngram_size = ? AND L1.type_lc = ? AND L2.type_lc = ? AND L3.type_lc = ? AND L4.type_lc = ?", \
                            (ngram_size, ngram[0], ngram[1], ngram[2], ngram[3]))
        elif ngram_size == 5:
            coca_c.execute("SELECT freq FROM ngrams \
                    JOIN lexicon AS L1 ON ngrams.wordoneID = L1.ID \
                    JOIN lexicon AS L2 ON ngrams.wordtwoID = L2.ID \
                    JOIN lexicon AS L3 ON ngrams.wordthreeID = L3.ID \
                    JOIN lexicon AS L4 ON ngrams.wordfourID = L4.ID \
                    JOIN lexicon AS L5 ON ngrams.wordfiveID = L5.ID \
                    WHERE ngram_size = ? AND L1.type_lc = ? AND L2.type_lc = ? AND L3.type_lc = ? AND L4.type_lc = ? AND L5.type_lc = ?", \
                            (ngram_size, ngram[0], ngram[1], ngram[2], ngram[3], ngram[4]))
        try:
            freq = coca_c.fetchone()[0]
        except:
            freq = 0.0000000000000001    # 15 zeros after the decimal point
        genre_counts[genre] = freq                
        genre_counts["total"] += freq 
        coca_conn.close()
    return genre_counts

#########################################

erf_levels = ((1,"one"),(2,"two"),(3,"three"),(4,"four"),(5,"five"),(6,"six"))
# 2-grams in GRC total
output_headings = "fic_tot, erf_level, ngram, freq_GRC, tokens_GRC, normFreq_GRC, freq_COCA, tokens_COCA, normFreq_COCA, keyness(LL), %DIFF\n"
output_file = 'Output\\RQ6_grc_2gram_level_lists.csv'
with open(output_file, 'w', encoding='utf-8') as f:
    f.write(output_headings)
    for num,text in erf_levels:
        num_str = str(num)
        column = "erf_" + text + "_freq"
        grc_c.execute("SELECT COUNT(*) FROM corpus WHERE erf_level = ?", (num,))
        tokens_GRC = grc_c.fetchone()[0]
        min_freq = 20 * (tokens_GRC/1000000)
        query = "SELECT L1.type_lc || '_' || L2.type_lc AS ngram, {} AS freq \
                FROM ngrams \
                JOIN lexicon AS L1 ON ngrams.wordoneID = L1.ID \
                JOIN lexicon AS L2 ON ngrams.wordtwoID = L2.ID \
                WHERE ngram_size = 2 AND freq >= ? \
                ORDER BY freq DESC".format(column)
        grc_c.execute(query, (min_freq,))
        print("Number of 2-grams in GRC total ERF Level", num, ":", len(grc_c.fetchall()))
        grc_c.execute(query, (min_freq,))
        list = grc_c.fetchall()
        for row in list:
            genre_counts = get_coca_info(row[0], 2)
            normFreq_GRC = row[1] * (1000000/tokens_GRC)
            tokens_COCA = 1139472212
            freq_COCA = genre_counts["total"]
            normFreq_COCA = freq_COCA * (1000000/tokens_COCA)
            expected_GRC_freq = (tokens_GRC * (row[1] + freq_COCA))/(tokens_GRC + tokens_COCA)
            expected_COCA_freq = (tokens_COCA * (row[1] + freq_COCA))/(tokens_GRC + tokens_COCA) 
            keyness = 2 * ((row[1]*math.log(row[1]/expected_GRC_freq)) + (freq_COCA*math.log(freq_COCA/expected_COCA_freq)))
            percent_diff = ((normFreq_GRC - normFreq_COCA)/normFreq_COCA) * 100
            f.write('TOTAL' + ',' + str(num_str) + ',' + ','.join([str(x) for x in row]) + ',' + str(tokens_GRC) + ',' + str(normFreq_GRC) + \
        ',' + str(freq_COCA) + ',' + str(tokens_COCA) + ',' + str(normFreq_COCA) + ',' + str(keyness) + ',' + str(percent_diff) + '\n')


        # 2-grams in GRC fiction
        grc_c.execute("SELECT COUNT(*) FROM corpus WHERE erf_level = ? AND f_nf = 'F'", (num,))
        tokens_GRC = grc_c.fetchone()[0]
        min_freq = 20 * (tokens_GRC/1000000)
        column = "erf_" + text + "_fic_freq"
        query = "SELECT L1.type_lc || '_' || L2.type_lc AS ngram, {} AS freq \
                FROM ngrams \
                JOIN lexicon AS L1 ON ngrams.wordoneID = L1.ID \
                JOIN lexicon AS L2 ON ngrams.wordtwoID = L2.ID \
                WHERE ngram_size = 2 AND freq >= ? \
                ORDER BY freq DESC".format(column)
        grc_c.execute(query, (min_freq,))
        print("Number of 2-grams in GRC fiction ERF Level", num, ":", len(grc_c.fetchall()))
        grc_c.execute(query, (min_freq,))
        for row in grc_c.fetchall():
            genre_counts = get_coca_info(row[0], 2)
            normFreq_GRC = row[1] * (1000000/tokens_GRC)
            tokens_COCA = 136217798
            freq_COCA = genre_counts["fic"]
            normFreq_COCA = freq_COCA * (1000000/tokens_COCA)
            expected_GRC_freq = (tokens_GRC * (row[1] + freq_COCA))/(tokens_GRC + tokens_COCA)
            expected_COCA_freq = (tokens_COCA * (row[1] + freq_COCA))/(tokens_GRC + tokens_COCA) 
            keyness = 2 * ((row[1]*math.log(row[1]/expected_GRC_freq)) + (freq_COCA*math.log(freq_COCA/expected_COCA_freq)))
            percent_diff = ((normFreq_GRC - normFreq_COCA)/normFreq_COCA) * 100
            f.write('FIC' + ',' + str(num_str) + ',' + ','.join([str(x) for x in row]) + ',' + str(tokens_GRC) + ',' + str(normFreq_GRC) + \
        ',' + str(freq_COCA) + ',' + str(tokens_COCA) + ',' + str(normFreq_COCA) + ',' + str(keyness) + ',' + str(percent_diff) + '\n')

# 3-grams in GRC total
output_file = 'Output\\RQ6_grc_3gram_level_lists.csv'
with open(output_file, 'w', encoding='utf-8') as f:
    f.write(output_headings)
    for num,text in erf_levels:
        num_str = str(num)
        column = "erf_" + text + "_freq"
        grc_c.execute("SELECT COUNT(*) FROM corpus WHERE erf_level = ?", (num,))
        tokens_GRC = grc_c.fetchone()[0]
        min_freq = 20 * (tokens_GRC/1000000)
        query = "SELECT L1.type_lc || '_' || L2.type_lc || '_' || L3.type_lc AS ngram, {} AS freq \
                FROM ngrams \
                JOIN lexicon AS L1 ON ngrams.wordoneID = L1.ID \
                JOIN lexicon AS L2 ON ngrams.wordtwoID = L2.ID \
                JOIN lexicon AS L3 ON ngrams.wordthreeID = L3.ID \
                WHERE ngram_size = 3 AND freq >= ? \
                ORDER BY freq DESC".format(column)
        grc_c.execute(query, (min_freq,))
        print("Number of 3-grams in GRC total ERF Level", num, ":", len(grc_c.fetchall()))
        grc_c.execute(query, (min_freq,))
        list = grc_c.fetchall()
        for row in list:
            genre_counts = get_coca_info(row[0], 2)
            normFreq_GRC = row[1] * (1000000/tokens_GRC)
            tokens_COCA = 1139472212
            freq_COCA = genre_counts["total"]
            normFreq_COCA = freq_COCA * (1000000/tokens_COCA)
            expected_GRC_freq = (tokens_GRC * (row[1] + freq_COCA))/(tokens_GRC + tokens_COCA)
            expected_COCA_freq = (tokens_COCA * (row[1] + freq_COCA))/(tokens_GRC + tokens_COCA) 
            keyness = 2 * ((row[1]*math.log(row[1]/expected_GRC_freq)) + (freq_COCA*math.log(freq_COCA/expected_COCA_freq)))
            percent_diff = ((normFreq_GRC - normFreq_COCA)/normFreq_COCA) * 100
            f.write('TOTAL' + ',' + str(num_str) + ',' + ','.join([str(x) for x in row]) + ',' + str(tokens_GRC) + ',' + str(normFreq_GRC) + \
        ',' + str(freq_COCA) + ',' + str(tokens_COCA) + ',' + str(normFreq_COCA) + ',' + str(keyness) + ',' + str(percent_diff) + '\n')


        # 3-grams in GRC fiction
        grc_c.execute("SELECT COUNT(*) FROM corpus WHERE erf_level = ? AND f_nf = 'F'", (num,))
        tokens_GRC = grc_c.fetchone()[0]
        min_freq = 20 * (tokens_GRC/1000000)
        column = "erf_" + text + "_fic_freq"
        query = "SELECT L1.type_lc || '_' || L2.type_lc || '_' || L3.type_lc AS ngram, {} AS freq \
                FROM ngrams \
                JOIN lexicon AS L1 ON ngrams.wordoneID = L1.ID \
                JOIN lexicon AS L2 ON ngrams.wordtwoID = L2.ID \
                JOIN lexicon AS L3 ON ngrams.wordthreeID = L3.ID \
                WHERE ngram_size = 3 AND freq >= ? \
                ORDER BY freq DESC".format(column)
        grc_c.execute(query, (min_freq,))
        print("Number of 3-grams in GRC fiction ERF Level", num, ":", len(grc_c.fetchall()))
        grc_c.execute(query, (min_freq,))
        for row in grc_c.fetchall():
            genre_counts = get_coca_info(row[0], 2)
            normFreq_GRC = row[1] * (1000000/tokens_GRC)
            tokens_COCA = 136217798
            freq_COCA = genre_counts["fic"]
            normFreq_COCA = freq_COCA * (1000000/tokens_COCA)
            expected_GRC_freq = (tokens_GRC * (row[1] + freq_COCA))/(tokens_GRC + tokens_COCA)
            expected_COCA_freq = (tokens_COCA * (row[1] + freq_COCA))/(tokens_GRC + tokens_COCA) 
            keyness = 2 * ((row[1]*math.log(row[1]/expected_GRC_freq)) + (freq_COCA*math.log(freq_COCA/expected_COCA_freq)))
            percent_diff = ((normFreq_GRC - normFreq_COCA)/normFreq_COCA) * 100
            f.write('FIC' + ',' + str(num_str) + ',' + ','.join([str(x) for x in row]) + ',' + str(tokens_GRC) + ',' + str(normFreq_GRC) + \
        ',' + str(freq_COCA) + ',' + str(tokens_COCA) + ',' + str(normFreq_COCA) + ',' + str(keyness) + ',' + str(percent_diff) + '\n')



# 4-grams in GRC total
output_file = 'Output\\RQ6_grc_4gram_level_lists.csv'
with open(output_file, 'w', encoding='utf-8') as f:
    f.write(output_headings)
    for num,text in erf_levels:
        num_str = str(num)
        column = "erf_" + text + "_freq"
        grc_c.execute("SELECT COUNT(*) FROM corpus WHERE erf_level = ?", (num,))
        tokens_GRC = grc_c.fetchone()[0]
        min_freq = 20 * (tokens_GRC/1000000)
        query = "SELECT L1.type_lc || '_' || L2.type_lc || '_' || L3.type_lc || '_' || L4.type_lc AS ngram, {} AS freq \
                FROM ngrams \
                JOIN lexicon AS L1 ON ngrams.wordoneID = L1.ID \
                JOIN lexicon AS L2 ON ngrams.wordtwoID = L2.ID \
                JOIN lexicon AS L3 ON ngrams.wordthreeID = L3.ID \
                JOIN lexicon AS L4 ON ngrams.wordfourID = L4.ID \
                WHERE ngram_size = 4 AND freq >= ? \
                ORDER BY freq DESC".format(column)
        grc_c.execute(query, (min_freq,))
        print("Number of 4-grams in GRC total ERF Level", num, ":", len(grc_c.fetchall()))
        grc_c.execute(query, (min_freq,))
        list = grc_c.fetchall()
        for row in list:
            genre_counts = get_coca_info(row[0], 2)
            normFreq_GRC = row[1] * (1000000/tokens_GRC)
            tokens_COCA = 1139472212
            freq_COCA = genre_counts["total"]
            normFreq_COCA = freq_COCA * (1000000/tokens_COCA)
            expected_GRC_freq = (tokens_GRC * (row[1] + freq_COCA))/(tokens_GRC + tokens_COCA)
            expected_COCA_freq = (tokens_COCA * (row[1] + freq_COCA))/(tokens_GRC + tokens_COCA) 
            keyness = 2 * ((row[1]*math.log(row[1]/expected_GRC_freq)) + (freq_COCA*math.log(freq_COCA/expected_COCA_freq)))
            percent_diff = ((normFreq_GRC - normFreq_COCA)/normFreq_COCA) * 100
            f.write('TOTAL' + ',' + str(num_str) + ',' + ','.join([str(x) for x in row]) + ',' + str(tokens_GRC) + ',' + str(normFreq_GRC) + \
        ',' + str(freq_COCA) + ',' + str(tokens_COCA) + ',' + str(normFreq_COCA) + ',' + str(keyness) + ',' + str(percent_diff) + '\n')


        # 4-grams in GRC fiction
        grc_c.execute("SELECT COUNT(*) FROM corpus WHERE erf_level = ? AND f_nf = 'F'", (num,))
        tokens_GRC = grc_c.fetchone()[0]
        min_freq = 20 * (tokens_GRC/1000000)
        column = "erf_" + text + "_fic_freq"
        query = "SELECT L1.type_lc || '_' || L2.type_lc || '_' || L3.type_lc || '_' || L4.type_lc AS ngram, {} AS freq \
                FROM ngrams \
                JOIN lexicon AS L1 ON ngrams.wordoneID = L1.ID \
                JOIN lexicon AS L2 ON ngrams.wordtwoID = L2.ID \
                JOIN lexicon AS L3 ON ngrams.wordthreeID = L3.ID \
                JOIN lexicon AS L4 ON ngrams.wordfourID = L4.ID \
                WHERE ngram_size = 4 AND freq >= ? \
                ORDER BY freq DESC".format(column)
        grc_c.execute(query, (min_freq,))
        print("Number of 4-grams in GRC fiction ERF Level", num, ":", len(grc_c.fetchall()))
        grc_c.execute(query, (min_freq,))
        for row in grc_c.fetchall():
            genre_counts = get_coca_info(row[0], 2)
            normFreq_GRC = row[1] * (1000000/tokens_GRC)
            tokens_COCA = 136217798
            freq_COCA = genre_counts["fic"]
            normFreq_COCA = freq_COCA * (1000000/tokens_COCA)
            expected_GRC_freq = (tokens_GRC * (row[1] + freq_COCA))/(tokens_GRC + tokens_COCA)
            expected_COCA_freq = (tokens_COCA * (row[1] + freq_COCA))/(tokens_GRC + tokens_COCA) 
            keyness = 2 * ((row[1]*math.log(row[1]/expected_GRC_freq)) + (freq_COCA*math.log(freq_COCA/expected_COCA_freq)))
            percent_diff = ((normFreq_GRC - normFreq_COCA)/normFreq_COCA) * 100
            f.write('FIC' + ',' + str(num_str) + ',' + ','.join([str(x) for x in row]) + ',' + str(tokens_GRC) + ',' + str(normFreq_GRC) + \
        ',' + str(freq_COCA) + ',' + str(tokens_COCA) + ',' + str(normFreq_COCA) + ',' + str(keyness) + ',' + str(percent_diff) + '\n')

# 5-grams in GRC total
output_file = 'Output\\RQ6_grc_5gram_level_lists.csv'
with open(output_file, 'w', encoding='utf-8') as f:
    f.write(output_headings)
    for num,text in erf_levels:
        num_str = str(num)
        column = "erf_" + text + "_freq"
        grc_c.execute("SELECT COUNT(*) FROM corpus WHERE erf_level = ?", (num,))
        tokens_GRC = grc_c.fetchone()[0]
        min_freq = 20 * (tokens_GRC/1000000)
        query = "SELECT L1.type_lc || '_' || L2.type_lc || '_' || L3.type_lc || '_' || L4.type_lc || '_' || L5.type_lc AS ngram, {} AS freq \
                FROM ngrams \
                JOIN lexicon AS L1 ON ngrams.wordoneID = L1.ID \
                JOIN lexicon AS L2 ON ngrams.wordtwoID = L2.ID \
                JOIN lexicon AS L3 ON ngrams.wordthreeID = L3.ID \
                JOIN lexicon AS L4 ON ngrams.wordfourID = L4.ID \
                JOIN lexicon AS L5 ON ngrams.wordfiveID = L5.ID \
                WHERE ngram_size = 5 AND freq >= ? \
                ORDER BY freq DESC".format(column)
        grc_c.execute(query, (min_freq,))
        print("Number of 5-grams in GRC total ERF Level", num, ":", len(grc_c.fetchall()))
        grc_c.execute(query, (min_freq,))
        list = grc_c.fetchall()
        for row in list:
            genre_counts = get_coca_info(row[0], 2)
            normFreq_GRC = row[1] * (1000000/tokens_GRC)
            tokens_COCA = 1139472212
            freq_COCA = genre_counts["total"]
            normFreq_COCA = freq_COCA * (1000000/tokens_COCA)
            expected_GRC_freq = (tokens_GRC * (row[1] + freq_COCA))/(tokens_GRC + tokens_COCA)
            expected_COCA_freq = (tokens_COCA * (row[1] + freq_COCA))/(tokens_GRC + tokens_COCA) 
            keyness = 2 * ((row[1]*math.log(row[1]/expected_GRC_freq)) + (freq_COCA*math.log(freq_COCA/expected_COCA_freq)))
            percent_diff = ((normFreq_GRC - normFreq_COCA)/normFreq_COCA) * 100
            f.write('TOTAL' + ',' + str(num_str) + ',' + ','.join([str(x) for x in row]) + ',' + str(tokens_GRC) + ',' + str(normFreq_GRC) + \
        ',' + str(freq_COCA) + ',' + str(tokens_COCA) + ',' + str(normFreq_COCA) + ',' + str(keyness) + ',' + str(percent_diff) + '\n')


        # 5-grams in GRC fiction
        grc_c.execute("SELECT COUNT(*) FROM corpus WHERE erf_level = ? AND f_nf = 'F'", (num,))
        tokens_GRC = grc_c.fetchone()[0]
        min_freq = 20 * (tokens_GRC/1000000)
        column = "erf_" + text + "_fic_freq"
        query = "SELECT L1.type_lc || '_' || L2.type_lc || '_' || L3.type_lc || '_' || L4.type_lc || '_' || L5.type_lc AS ngram, {} AS freq \
                FROM ngrams \
                JOIN lexicon AS L1 ON ngrams.wordoneID = L1.ID \
                JOIN lexicon AS L2 ON ngrams.wordtwoID = L2.ID \
                JOIN lexicon AS L3 ON ngrams.wordthreeID = L3.ID \
                JOIN lexicon AS L4 ON ngrams.wordfourID = L4.ID \
                JOIN lexicon AS L5 ON ngrams.wordfiveID = L5.ID \
                WHERE ngram_size = 5 AND freq >= ? \
                ORDER BY freq DESC".format(column)
        grc_c.execute(query, (min_freq,))
        print("Number of 5-grams in GRC fiction ERF Level", num, ":", len(grc_c.fetchall()))
        grc_c.execute(query, (min_freq,))
        for row in grc_c.fetchall():
            genre_counts = get_coca_info(row[0], 2)
            normFreq_GRC = row[1] * (1000000/tokens_GRC)
            tokens_COCA = 136217798
            freq_COCA = genre_counts["fic"]
            normFreq_COCA = freq_COCA * (1000000/tokens_COCA)
            expected_GRC_freq = (tokens_GRC * (row[1] + freq_COCA))/(tokens_GRC + tokens_COCA)
            expected_COCA_freq = (tokens_COCA * (row[1] + freq_COCA))/(tokens_GRC + tokens_COCA) 
            keyness = 2 * ((row[1]*math.log(row[1]/expected_GRC_freq)) + (freq_COCA*math.log(freq_COCA/expected_COCA_freq)))
            percent_diff = ((normFreq_GRC - normFreq_COCA)/normFreq_COCA) * 100
            f.write('FIC' + ',' + str(num_str) + ',' + ','.join([str(x) for x in row]) + ',' + str(tokens_GRC) + ',' + str(normFreq_GRC) + \
        ',' + str(freq_COCA) + ',' + str(tokens_COCA) + ',' + str(normFreq_COCA) + ',' + str(keyness) + ',' + str(percent_diff) + '\n')


grc_conn.commit()
grc_conn.close()

Number of 2-grams in GRC total ERF Level 1 : 5531
Number of 2-grams in GRC fiction ERF Level 1 : 5478
Number of 2-grams in GRC total ERF Level 2 : 4910
Number of 2-grams in GRC fiction ERF Level 2 : 4869
Number of 2-grams in GRC total ERF Level 3 : 4878
Number of 2-grams in GRC fiction ERF Level 3 : 4948
Number of 2-grams in GRC total ERF Level 4 : 4681
Number of 2-grams in GRC fiction ERF Level 4 : 4741
Number of 2-grams in GRC total ERF Level 5 : 4694
Number of 2-grams in GRC fiction ERF Level 5 : 4851
Number of 2-grams in GRC total ERF Level 6 : 4342
Number of 2-grams in GRC fiction ERF Level 6 : 4856
Number of 3-grams in GRC total ERF Level 1 : 2417
Number of 3-grams in GRC fiction ERF Level 1 : 2399
Number of 3-grams in GRC total ERF Level 2 : 1678
Number of 3-grams in GRC fiction ERF Level 2 : 1750
Number of 3-grams in GRC total ERF Level 3 : 1481
Number of 3-grams in GRC fiction ERF Level 3 : 1590
Number of 3-grams in GRC total ERF Level 4 : 1307
Number of 3-grams in GRC fiction

In [14]:

# COCA 2grams - fiction

coca_filename = 'D:\\coca_fic.sqlite'
coca_conn = sqlite3.connect(coca_filename)
coca_c = coca_conn.cursor()

output_headings = "word1, word2, freq\n"
output_file = 'Output\\RQ6_coca_fic_2gram.csv'
with open(output_file, 'w') as f:
    f.write(output_headings)

    coca_c.execute("SELECT ID FROM lexicon WHERE type_lc = '@@@@@@@@@@'")
    typeID = coca_c.fetchone()[0]
    coca_c.execute("SELECT COUNT(*) FROM corpus WHERE word_typeID != ?", (typeID,))
    total_tokens = coca_c.fetchone()[0]
    min_freq = 20 * (total_tokens / 1000000)
    coca_c.execute("SELECT L1.type_lc AS wordone, L2.type_lc AS wordtwo, freq \
                FROM ngrams \
                JOIN lexicon AS L1 ON ngrams.wordoneID = L1.ID \
                JOIN lexicon AS L2 ON ngrams.wordtwoID = L2.ID \
                WHERE ngram_size = 2 AND freq >= ? \
                AND wordoneID != ? AND wordtwoID != ? \
                ORDER BY freq DESC", (min_freq,typeID,typeID))
    
    for row in coca_c.fetchall():
        f.write(','.join([str(x) for x in row]) + '\n')
    
    
    coca_conn.close()


In [2]:

# COCA 2grams-total
import sqlite3

# create dictionary of ngrams where key is ngram and value is frequency
ngrams = {}
genres = ("acad", "blog", "fic", "mag", "news", "spok", "tvm", "web")

for genre in genres:
    print("Processing genre:", genre)
    coca_filename = 'D:\\coca_' + genre + '.sqlite'
    coca_conn = sqlite3.connect(coca_filename)
    coca_c = coca_conn.cursor()
    
    coca_c.execute("CREATE INDEX IF NOT EXISTS ngram_count_IN ON ngrams (ngram_size, freq)")
    coca_conn.commit()

    coca_c.execute("SELECT ID FROM lexicon WHERE type_lc = '@@@@@@@@@@'")
    typeID = coca_c.fetchone()[0]
    coca_c.execute("SELECT COUNT(*) FROM corpus WHERE word_typeID != ?", (typeID,))
    total_tokens = coca_c.fetchone()[0]
    min_freq = 5 * (total_tokens / 1000000)
    
    coca_c.execute("SELECT L1.type_lc || '_' || L2.type_lc AS word, freq \
                    FROM ngrams \
                    JOIN lexicon AS L1 ON ngrams.wordoneID = L1.ID \
                    JOIN lexicon AS L2 ON ngrams.wordtwoID = L2.ID \
                    WHERE ngram_size = 2 AND freq >= ? \
                    AND wordoneID != ? AND wordtwoID != ? \
                    ORDER BY freq DESC", (min_freq, typeID, typeID))

    for row in coca_c.fetchall():
        # if ngram is already in dictionary, add the frequency to the existing value
        if row[0] in ngrams:
            ngrams[row[0]] += row[1]
        # if ngram is not in dictionary, add it with the frequency as the value
        else:
            ngrams[row[0]] = row[1]

    coca_conn.close()

# arrange the dictionary in descending frequency order
sorted_ngrams = sorted(ngrams.items(), key=lambda x: x[1], reverse=True)
output_file = 'Output\\RQ6_coca_tot_2gram.csv'
output_headings = "ngram, freq\n"

# write the dictionary to a csv file
with open(output_file, 'w', encoding='utf-8') as f:
    f.write(output_headings)
    for ngram in sorted_ngrams:
        if ngram[1] >= 22789: # 20 per million based on COCA total count
            f.write(str(ngram[0]) + ',' + str(ngram[1]) + '\n')




Processing genre: acad
Processing genre: blog
Processing genre: fic
Processing genre: mag
Processing genre: news
Processing genre: spok
Processing genre: tvm
Processing genre: web


In [3]:
import sqlite3

# COCA 3 grams

coca_filename = 'D:\\coca_fic.sqlite'
coca_conn = sqlite3.connect(coca_filename)
coca_c = coca_conn.cursor()

output_headings = "word1, word2, word3, freq\n"
output_file = 'Output\\RQ6_coca_fic_3gram.csv'
with open(output_file, 'w') as f:
    f.write(output_headings)

    coca_c.execute("SELECT ID FROM lexicon WHERE type_lc = '@@@@@@@@@@'")
    typeID = coca_c.fetchone()[0]
    coca_c.execute("SELECT COUNT(*) FROM corpus WHERE word_typeID != ?", (typeID,))
    total_tokens = coca_c.fetchone()[0]
    min_freq = 20 * (total_tokens / 1000000)
    coca_c.execute("SELECT L1.type_lc AS wordone, L2.type_lc AS wordtwo, L3.type_lc AS wordthree, freq \
                FROM ngrams \
                JOIN lexicon AS L1 ON ngrams.wordoneID = L1.ID \
                JOIN lexicon AS L2 ON ngrams.wordtwoID = L2.ID \
                JOIN lexicon AS L3 ON ngrams.wordthreeID = L3.ID \
                WHERE ngram_size = 3 AND freq >= ? \
                AND wordoneID != ? AND wordtwoID != ? AND wordthreeID != ? \
                ORDER BY freq DESC", (min_freq,typeID,typeID,typeID))
    
    for row in coca_c.fetchall():
        f.write(','.join([str(x) for x in row]) + '\n')
    
    
    coca_conn.close()


# create dictionary of ngrams where key is ngram and value is frequency
ngrams = {}
genres = ("acad", "blog", "fic", "mag", "news", "spok", "tvm", "web")

for genre in genres:
    print("Processing genre:", genre)
    coca_filename = 'D:\\coca_' + genre + '.sqlite'
    coca_conn = sqlite3.connect(coca_filename)
    coca_c = coca_conn.cursor()
    
    coca_c.execute("CREATE INDEX IF NOT EXISTS ngram_count_IN ON ngrams (ngram_size, freq)")
    coca_conn.commit()

    coca_c.execute("SELECT ID FROM lexicon WHERE type_lc = '@@@@@@@@@@'")
    typeID = coca_c.fetchone()[0]
    coca_c.execute("SELECT COUNT(*) FROM corpus WHERE word_typeID != ?", (typeID,))
    total_tokens = coca_c.fetchone()[0]
    min_freq = 5 * (total_tokens / 1000000)
    
    coca_c.execute("SELECT L1.type_lc || '_' || L2.type_lc || '_' || L3.type_lc AS word, freq \
                    FROM ngrams \
                    JOIN lexicon AS L1 ON ngrams.wordoneID = L1.ID \
                    JOIN lexicon AS L2 ON ngrams.wordtwoID = L2.ID \
                    JOIN lexicon AS L3 ON ngrams.wordthreeID = L3.ID \
                    WHERE ngram_size = 3 AND freq >= ? \
                    AND wordoneID != ? AND wordtwoID != ? AND wordthreeID != ? \
                    ORDER BY freq DESC", (min_freq, typeID, typeID, typeID))


    for row in coca_c.fetchall():
        # if ngram is already in dictionary, add the frequency to the existing value
        if row[0] in ngrams:
            ngrams[row[0]] += row[1]
        # if ngram is not in dictionary, add it with the frequency as the value
        else:
            ngrams[row[0]] = row[1]

    coca_conn.close()

# arrange the dictionary in descending frequency order
sorted_ngrams = sorted(ngrams.items(), key=lambda x: x[1], reverse=True)
output_file = 'Output\\RQ6_coca_tot_3gram.csv'
output_headings = "ngram, freq\n"

# write the dictionary to a csv file
with open(output_file, 'w', encoding='utf-8') as f:
    f.write(output_headings)
    for ngram in sorted_ngrams:
        if ngram[1] >= 22789: # 20 per million based on COCA total count
            f.write(str(ngram[0]) + ',' + str(ngram[1]) + '\n')

Processing genre: acad
Processing genre: blog
Processing genre: fic
Processing genre: mag
Processing genre: news
Processing genre: spok
Processing genre: tvm
Processing genre: web


In [6]:
import sqlite3

# COCA 4 grams

coca_filename = 'D:\\coca_fic.sqlite'
coca_conn = sqlite3.connect(coca_filename)
coca_c = coca_conn.cursor()

output_headings = "word1, word2, word3, word4, freq\n"
output_file = 'Output\\RQ6_coca_fic_4gram.csv'
with open(output_file, 'w') as f:
    f.write(output_headings)

    coca_c.execute("SELECT ID FROM lexicon WHERE type_lc = '@@@@@@@@@@'")
    typeID = coca_c.fetchone()[0]
    coca_c.execute("SELECT COUNT(*) FROM corpus WHERE word_typeID != ?", (typeID,))
    total_tokens = coca_c.fetchone()[0]
    min_freq = 20 * (total_tokens / 1000000)
    coca_c.execute("SELECT L1.type_lc AS wordone, L2.type_lc AS wordtwo, L3.type_lc AS wordthree, \
                   L4.type_lc AS wordfour, freq \
                FROM ngrams \
                JOIN lexicon AS L1 ON ngrams.wordoneID = L1.ID \
                JOIN lexicon AS L2 ON ngrams.wordtwoID = L2.ID \
                JOIN lexicon AS L3 ON ngrams.wordthreeID = L3.ID \
                JOIN lexicon AS L4 ON ngrams.wordfourID = L4.ID \
                WHERE ngram_size = 4 AND freq >= ? \
                AND wordoneID != ? AND wordtwoID != ? AND wordthreeID != ? AND wordfourID != ? \
                ORDER BY freq DESC", (min_freq,typeID,typeID,typeID,typeID))
    
    for row in coca_c.fetchall():
        f.write(','.join([str(x) for x in row]) + '\n')
    
    
    coca_conn.close()


# create dictionary of ngrams where key is ngram and value is frequency
ngrams = {}
genres = ("acad", "blog", "fic", "mag", "news", "spok", "tvm", "web")

for genre in genres:
    print("Processing genre:", genre)
    coca_filename = 'D:\\coca_' + genre + '.sqlite'
    coca_conn = sqlite3.connect(coca_filename)
    coca_c = coca_conn.cursor()
    
    coca_c.execute("CREATE INDEX IF NOT EXISTS ngram_count_IN ON ngrams (ngram_size, freq)")
    coca_conn.commit()

    coca_c.execute("SELECT ID FROM lexicon WHERE type_lc = '@@@@@@@@@@'")
    typeID = coca_c.fetchone()[0]
    coca_c.execute("SELECT COUNT(*) FROM corpus WHERE word_typeID != ?", (typeID,))
    total_tokens = coca_c.fetchone()[0]
    min_freq = 5 * (total_tokens / 1000000)
    
    coca_c.execute("SELECT L1.type_lc || '_' || L2.type_lc || '_' || L3.type_lc || '_' || L4.type_lc AS word, freq \
                    FROM ngrams \
                    JOIN lexicon AS L1 ON ngrams.wordoneID = L1.ID \
                    JOIN lexicon AS L2 ON ngrams.wordtwoID = L2.ID \
                    JOIN lexicon AS L3 ON ngrams.wordthreeID = L3.ID \
                    JOIN lexicon AS L4 ON ngrams.wordfourID = L4.ID \
                    WHERE ngram_size = 4 AND freq >= ? \
                    AND wordoneID != ? AND wordtwoID != ? AND wordthreeID != ? AND wordfourID != ? \
                    ORDER BY freq DESC", (min_freq, typeID, typeID, typeID, typeID))


    for row in coca_c.fetchall():
        # if ngram is already in dictionary, add the frequency to the existing value
        if row[0] in ngrams:
            ngrams[row[0]] += row[1]
        # if ngram is not in dictionary, add it with the frequency as the value
        else:
            ngrams[row[0]] = row[1]

    coca_conn.close()

# arrange the dictionary in descending frequency order
sorted_ngrams = sorted(ngrams.items(), key=lambda x: x[1], reverse=True)
output_file = 'Output\\RQ6_coca_tot_4gram.csv'
output_headings = "ngram, freq\n"

# write the dictionary to a csv file
with open(output_file, 'w', encoding='utf-8') as f:
    f.write(output_headings)
    for ngram in sorted_ngrams:
        if ngram[1] >= 22789: # 20 per million based on COCA total count
            f.write(str(ngram[0]) + ',' + str(ngram[1]) + '\n')

Processing genre: acad
Processing genre: blog
Processing genre: fic
Processing genre: mag
Processing genre: news
Processing genre: spok
Processing genre: tvm
Processing genre: web


In [7]:
import sqlite3

# COCA 5 grams

coca_filename = 'D:\\coca_fic.sqlite'
coca_conn = sqlite3.connect(coca_filename)
coca_c = coca_conn.cursor()

output_headings = "word1, word2, word3, word4, word5, freq\n"
output_file = 'Output\\RQ6_coca_fic_5gram.csv'
with open(output_file, 'w') as f:
    f.write(output_headings)

    coca_c.execute("SELECT ID FROM lexicon WHERE type_lc = '@@@@@@@@@@'")
    typeID = coca_c.fetchone()[0]
    coca_c.execute("SELECT COUNT(*) FROM corpus WHERE word_typeID != ?", (typeID,))
    total_tokens = coca_c.fetchone()[0]
    min_freq = 20 * (total_tokens / 1000000)
    coca_c.execute("SELECT L1.type_lc AS wordone, L2.type_lc AS wordtwo, L3.type_lc AS wordthree, \
                   L4.type_lc AS wordfour, L5.type_lc AS wordfive, freq \
                FROM ngrams \
                JOIN lexicon AS L1 ON ngrams.wordoneID = L1.ID \
                JOIN lexicon AS L2 ON ngrams.wordtwoID = L2.ID \
                JOIN lexicon AS L3 ON ngrams.wordthreeID = L3.ID \
                JOIN lexicon AS L4 ON ngrams.wordfourID = L4.ID \
                JOIN lexicon AS L5 ON ngrams.wordfiveID = L5.ID \
                WHERE ngram_size = 5 AND freq >= ? \
                AND wordoneID != ? AND wordtwoID != ? AND wordthreeID != ? AND wordfourID != ? AND wordfiveID != ? \
                ORDER BY freq DESC", (min_freq,typeID,typeID,typeID,typeID,typeID))
    
    for row in coca_c.fetchall():
        f.write(','.join([str(x) for x in row]) + '\n')
    
    
    coca_conn.close()


# create dictionary of ngrams where key is ngram and value is frequency
ngrams = {}
genres = ("acad", "blog", "fic", "mag", "news", "spok", "tvm", "web")

for genre in genres:
    print("Processing genre:", genre)
    coca_filename = 'D:\\coca_' + genre + '.sqlite'
    coca_conn = sqlite3.connect(coca_filename)
    coca_c = coca_conn.cursor()
    
    coca_c.execute("CREATE INDEX IF NOT EXISTS ngram_count_IN ON ngrams (ngram_size, freq)")
    coca_conn.commit()

    coca_c.execute("SELECT ID FROM lexicon WHERE type_lc = '@@@@@@@@@@'")
    typeID = coca_c.fetchone()[0]
    coca_c.execute("SELECT COUNT(*) FROM corpus WHERE word_typeID != ?", (typeID,))
    total_tokens = coca_c.fetchone()[0]
    min_freq = 5 * (total_tokens / 1000000)
    
    coca_c.execute("SELECT L1.type_lc || '_' || L2.type_lc || '_' || L3.type_lc || '_' || L4.type_lc || '_' || L5.type_lc AS word, freq \
                    FROM ngrams \
                    JOIN lexicon AS L1 ON ngrams.wordoneID = L1.ID \
                    JOIN lexicon AS L2 ON ngrams.wordtwoID = L2.ID \
                    JOIN lexicon AS L3 ON ngrams.wordthreeID = L3.ID \
                    JOIN lexicon AS L4 ON ngrams.wordfourID = L4.ID \
                    JOIN lexicon AS L5 ON ngrams.wordfiveID = L5.ID \
                    WHERE ngram_size = 5 AND freq >= ? \
                    AND wordoneID != ? AND wordtwoID != ? AND wordthreeID != ? AND wordfourID != ? AND wordfiveID != ? \
                    ORDER BY freq DESC", (min_freq, typeID, typeID, typeID, typeID, typeID))


    for row in coca_c.fetchall():
        # if ngram is already in dictionary, add the frequency to the existing value
        if row[0] in ngrams:
            ngrams[row[0]] += row[1]
        # if ngram is not in dictionary, add it with the frequency as the value
        else:
            ngrams[row[0]] = row[1]

    coca_conn.close()

# arrange the dictionary in descending frequency order
sorted_ngrams = sorted(ngrams.items(), key=lambda x: x[1], reverse=True)
output_file = 'Output\\RQ6_coca_tot_5gram.csv'
output_headings = "ngram, freq\n"

# write the dictionary to a csv file
with open(output_file, 'w', encoding='utf-8') as f:
    f.write(output_headings)
    for ngram in sorted_ngrams:
        if ngram[1] >= 22789: # 20 per million based on COCA total count
            f.write(str(ngram[0]) + ',' + str(ngram[1]) + '\n')

Processing genre: acad
Processing genre: blog
Processing genre: fic
Processing genre: mag
Processing genre: news
Processing genre: spok
Processing genre: tvm
Processing genre: web


In [2]:
import sqlite3

grc_conn = sqlite3.connect('D:\\graded_readers_mwe.sqlite') # This creates the database file
grc_c = grc_conn.cursor() # Creates a cursor
grc_c.execute("CREATE INDEX IF NOT EXISTS ngram_count_IN ON ngrams (ngram_size, total_grc_freq)")
grc_c.execute("CREATE INDEX IF NOT EXISTS ngram_count_fic_IN ON ngrams (ngram_size, total_grc_fic_freq)")
grc_conn.commit()

print("creating indices")
grc_c.execute("CREATE INDEX IF NOT EXISTS ngrams_wordoneID_idx ON ngrams(wordoneID)")
grc_c.execute("CREATE INDEX IF NOT EXISTS ngrams_wordtwoID_idx ON ngrams(wordtwoID)")
grc_c.execute("CREATE INDEX IF NOT EXISTS ngrams_ngram_size_idx ON ngrams(ngram_size)")
grc_c.execute("CREATE INDEX IF NOT EXISTS corpus_word_typeID_idx ON corpus(word_typeID)")
grc_c.execute("CREATE INDEX IF NOT EXISTS corpus_f_nf_idx ON corpus(f_nf)")
grc_c.execute("CREATE INDEX IF NOT EXISTS corpus_erf_level_idx ON corpus(erf_level)")
grc_c.execute("CREATE INDEX IF NOT EXISTS corpus_file_id_idx ON corpus(file_id)")
grc_c.execute("CREATE INDEX corpus_f_nf_erf_level_idx ON corpus (f_nf, erf_level)")
grc_conn.commit()

grc_conn.commit()

print("updating total_grc_fic_freq")
grc_c.execute("""UPDATE ngrams
SET total_grc_fic_freq = (
    SELECT COUNT(*) 
    FROM corpus
    JOIN corpus AS corpus_2 ON corpus.file_id = corpus_2.file_id AND corpus.rowid + 1 = corpus_2.rowid
    WHERE ngrams.wordoneID = corpus.word_typeID AND ngrams.wordtwoID = corpus_2.word_typeID
    AND corpus.f_nf = 'F'
    AND ngrams.ngram_size = 2)"""
)

print("updating erf_one_fic_freq")
grc_c.execute("""UPDATE ngrams
SET erf_one_fic_freq = (
    SELECT COUNT(*)
    FROM corpus
    JOIN corpus AS corpus_2 ON corpus.file_id = corpus_2.file_id AND corpus.rowid + 1 = corpus_2.rowid
    WHERE ngrams.wordoneID = corpus.word_typeID AND ngrams.wordtwoID = corpus_2.word_typeID
    AND corpus.f_nf = 'F' AND corpus.erf_level = 1
    AND ngrams.ngram_size = 2)"""
)

print("updating erf_two_fic_freq")
grc_c.execute("""UPDATE ngrams
SET erf_two_fic_freq = (
    SELECT COUNT(*)
    FROM corpus
    JOIN corpus AS corpus_2 ON corpus.file_id = corpus_2.file_id AND corpus.rowid + 1 = corpus_2.rowid
    WHERE ngrams.wordoneID = corpus.word_typeID AND ngrams.wordtwoID = corpus_2.word_typeID
    AND corpus.f_nf = 'F' AND corpus.erf_level = 2
    AND ngrams.ngram_size = 2)"""
)

print("updating erf_three_fic_freq")
grc_c.execute("""UPDATE ngrams
SET erf_three_fic_freq = (
    SELECT COUNT(*)
    FROM corpus
    JOIN corpus AS corpus_2 ON corpus.file_id = corpus_2.file_id AND corpus.rowid + 1 = corpus_2.rowid
    WHERE ngrams.wordoneID = corpus.word_typeID AND ngrams.wordtwoID = corpus_2.word_typeID
    AND corpus.f_nf = 'F' AND corpus.erf_level = 3
    AND ngrams.ngram_size = 2)"""
)

print("updating erf_four_fic_freq")
grc_c.execute("""UPDATE ngrams
SET erf_four_fic_freq = (
    SELECT COUNT(*)
    FROM corpus
    JOIN corpus AS corpus_2 ON corpus.file_id = corpus_2.file_id AND corpus.rowid + 1 = corpus_2.rowid
    WHERE ngrams.wordoneID = corpus.word_typeID AND ngrams.wordtwoID = corpus_2.word_typeID
    AND corpus.f_nf = 'F' AND corpus.erf_level = 4
    AND ngrams.ngram_size = 2)"""
)

print("updating erf_five_fic_freq")
grc_c.execute("""UPDATE ngrams
SET erf_five_fic_freq = (
    SELECT COUNT(*)
    FROM corpus
    JOIN corpus AS corpus_2 ON corpus.file_id = corpus_2.file_id AND corpus.rowid + 1 = corpus_2.rowid
    WHERE ngrams.wordoneID = corpus.word_typeID AND ngrams.wordtwoID = corpus_2.word_typeID
    AND corpus.f_nf = 'F' AND corpus.erf_level = 5
    AND ngrams.ngram_size = 2)"""
)

print("updating erf_six_fic_freq")
grc_c.execute("""UPDATE ngrams
SET erf_six_fic_freq = (
    SELECT COUNT(*)
    FROM corpus
    JOIN corpus AS corpus_2 ON corpus.file_id = corpus_2.file_id AND corpus.rowid + 1 = corpus_2.rowid 
    WHERE ngrams.wordoneID = corpus.word_typeID AND ngrams.wordtwoID = corpus_2.word_typeID
    AND corpus.f_nf = 'F' AND corpus.erf_level = 6
    AND ngrams.ngram_size = 2)"""
)

grc_conn.commit()
grc_conn.close()

creating indices
updating total_grc_freq


In [11]:
# producing keyness information for coca files

import sqlite3, os

# connect to the database
if os.name == 'nt':  # 'nt' is the name for Windows in Python's os module
    grc_conn = sqlite3.connect('D:\\graded_readers_mwe.sqlite')
else:  # Assume Unix-based system (like macOS or Linux)
    try:
        grc_conn = sqlite3.connect('/Volumes/T7/graded_readers_mwe.sqlite')
    except:
        grc_conn = sqlite3.connect('/Volumes/KINGSTON/graded_readers_mwe.sqlite')

grc_c = grc_conn.cursor() # Creates a cursor

gramsizes = [(2,"two"),(3,"three"),(4,"four"),(5,"five")]
genres = ["tot", "fic"] 

# creating indices
print("creating indices: 2-gram")
grc_c.execute("""CREATE INDEX IF NOT EXISTS twogram_search ON ngrams(ngram_size, wordoneID, wordtwoID)""")
print("creating indices: 3-gram")
grc_c.execute("""CREATE INDEX IF NOT EXISTS threegram_search ON ngrams(ngram_size, wordoneID, wordtwoID, wordthreeID)""")
print("creating indices: 4-gram")
grc_c.execute("""CREATE INDEX IF NOT EXISTS fourgram_search ON ngrams(ngram_size, wordoneID, wordtwoID, wordthreeID, wordfourID)""")
print("creating indices: 5-gram")
grc_c.execute("""CREATE INDEX IF NOT EXISTS fivegram_search ON ngrams(ngram_size, wordoneID, wordtwoID, wordthreeID, wordfourID, wordfiveID)""")
grc_conn.commit()

for genre in genres:
    for num, text in gramsizes:
        filename = "RQ6_coca_" + genre + "_" + str(num) + "gram.csv"
        print("writing " + filename)
        filepath = os.path.join('Output', filename)
        with open(filepath, 'r', encoding='utf-8') as f:
            phrase_list = f.readlines()
        output_filename = "RQ6_coca_" + genre + "_" + str(num) + "gram_KEYNESS.csv"
        output_filepath = os.path.join('Output', output_filename)
        with open(output_filepath, 'w', encoding='utf-8') as f:
            f.write("ngram, coca_freq, grc_freq\n")
            counter = 0
            for line in phrase_list:
                counter += 1
                if counter == 1:
                    continue
                line = line.split(",")
                ngram = line[0].split("_")
                wordone = ngram[0].lower()
                try:
                    wordoneID = grc_c.execute("""SELECT ID FROM lexicon WHERE type_lc = ?""", (wordone,)).fetchone()[0]
                except:
                    print(ngram[0].lower(), "not found")
                    wordone = "ERROR"
                wordtwo = ngram[1].lower()
                try:
                    wordtwoID = grc_c.execute("""SELECT ID FROM lexicon WHERE type_lc = ?""", (wordtwo,)).fetchone()[0]
                except:
                    print(ngram[1].lower(), "not found")
                    wordtwo = "ERROR"
                query_string = """wordoneID = ? AND wordtwoID = ?"""
                if num >= 3:
                    wordthree = ngram[2].lower()
                    try:
                        wordthreeID = grc_c.execute("""SELECT ID FROM lexicon WHERE type_lc = ?""", (wordthree,)).fetchone()[0]
                    except:
                        print(ngram[2].lower(), "not found")
                        wordthree = "ERROR"
                    query_string = """wordoneID = ? AND wordtwoID = ? AND wordthreeID = ?"""
                if num >= 4:
                    wordfour = ngram[3].lower()
                    try:
                        wordfourID = grc_c.execute("""SELECT ID FROM lexicon WHERE type_lc = ?""", (wordfour,)).fetchone()[0]
                    except:
                        print(ngram[3].lower(), "not found")
                        wordfour = "ERROR"
                    query_string = """wordoneID = ? AND wordtwoID = ? AND wordthreeID = ? AND wordfourID = ?"""
                if num == 5:
                    wordfive = ngram[4].lower()
                    try:
                        wordfiveID = grc_c.execute("""SELECT ID FROM lexicon WHERE type_lc = ?""", (wordfive,)).fetchone()[0]
                    except:
                        print(ngram[4].lower(), "not found")
                        wordfive = "ERROR"
                    query_string = """wordoneID = ? AND wordtwoID = ? AND wordthreeID = ? AND wordfourID = ? AND wordfiveID = ?"""
                freq = line[1]
                if genre == "tot":
                    query = """SELECT total_grc_freq
                            FROM ngrams
                            WHERE ngram_size = ? AND """ + query_string
                else:
                    query = """SELECT total_grc_fic_freq
                            FROM ngrams
                            WHERE ngram_size = ? AND """ + query_string
                if num == 2:
                    grc_c.execute(query, (num, wordoneID, wordtwoID))
                elif num == 3:
                    grc_c.execute(query, (num, wordoneID, wordtwoID, wordthreeID))
                elif num == 4:
                    grc_c.execute(query, (num, wordoneID, wordtwoID, wordthreeID, wordfourID))
                elif num == 5:
                    grc_c.execute(query, (num, wordoneID, wordtwoID, wordthreeID, wordfourID, wordfiveID))
                try:
                    grc_freq = str(grc_c.fetchone()[0])
                except:
                    grc_freq = "0"
                f.write(line[0] + "," + freq.strip("\n") + "," + grc_freq + "\n")

grc_conn.close()


creating indices: 2-gram
creating indices: 3-gram
creating indices: 4-gram
creating indices: 5-gram
writing RQ6_coca_tot_2gram.csv
u.s. not found
qwq not found
amp not found
@(off not found
@(off not found
@(footage not found
@(voiceover not found
@!unidentified not found
writing RQ6_coca_tot_3gram.csv
@(off not found
u.s. not found
writing RQ6_coca_tot_4gram.csv
writing RQ6_coca_tot_5gram.csv
writing RQ6_coca_fic_2gram.csv
rsquo not found
int not found
writing RQ6_coca_fic_3gram.csv
int not found
writing RQ6_coca_fic_4gram.csv
writing RQ6_coca_fic_5gram.csv


In [12]:
import sqlite3, os

# connect to the database
if os.name == 'nt':  # 'nt' is the name for Windows in Python's os module
    grc_conn = sqlite3.connect('D:\\graded_readers_mwe.sqlite')
else:  # Assume Unix-based system (like macOS or Linux)
    try:
        grc_conn = sqlite3.connect('/Volumes/T7/0a_graded_readers.sqlite')
    except:
        grc_conn = sqlite3.connect('/Volumes/KINGSTON/0a_graded_readers.sqlite')

grc_c = grc_conn.cursor() # Creates a cursor

# search the two_grams view for type_1 and type_2
# return the 10 lines before and 10 lines after
# print the lines

type_1 = "all"
type_2 = "rights"

grc_c.execute("""SELECT L1.rowid 
                FROM corpus_table AS L1
                JOIN corpus_table AS L2 
                    ON L1.rowid = L2.rowid - 1
                WHERE L1.type_lc = ? AND L2.type_lc = ?""", (type_1, type_2))
rowid = grc_c.fetchone()[0]

grc_c.execute("""SELECT type_lc, publisher, title, erf_level FROM corpus_table WHERE rowid BETWEEN ? AND ?""", (rowid - 20, rowid + 20))
lines = grc_c.fetchall()

for line in lines:
    print(line)


grc_conn.close()

TypeError: 'NoneType' object is not subscriptable